# Loading Daily S&P 500 Data

In [1]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  
println("Number of days in the sample: ",T)

dN = Array{Date}(T)                      #convert to Date
for t = 1:T
    dN[t] = Date(x[t+1,1],"d/m/y")
end

Number of days in the sample: 9352


# Autocorrelations

In [2]:
using StatsBase

plags = 1:5
xCorr = autocor(R,plags)
println("\nlag autocorrr and its t-stat of excess returns")
println(round([plags xCorr sqrt(T)*xCorr],3))


lag autocorrr and its t-stat of excess returns
[1.0 -0.026 -2.48
 2.0 -0.044 -4.208
 3.0 -0.007 -0.684
 4.0 -0.022 -2.124
 5.0 -0.018 -1.744]


# Autoregressions

In [3]:
function OlsFn(y,x)                      #OLS estimation 
    T = size(y,1)                        #alternative: use GLM.jl
    #n = size(y,2)
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)*(T-1)/T
    Covb   = kron(Covres,inv(x'x))
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2a,T           #the outputs
end

OlsFn (generic function with 1 method)

In [4]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb,1) R[1:end-1]]
(b,res,yhat,Covb,R2a,) = OlsFn(y,x) 
Stdb  = sqrt(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
println(round([b tstat],3))

Results from AR(1): intercept and slope with t-stats
[0.039 3.406
 -0.026 -2.481]


In [5]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb,1) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,R2a,) = OlsFn(y,x) 
Stdb  = sqrt(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos]with t-stats")
println(round([b tstat],3))

Results from AR(1) with dummies: [intercept, slope neg, slope pos]with t-stats
[0.001 0.058
 -0.075 -4.395
 0.027 1.518]


#    # Recursive Estimation and Out-of-Sample R2

In [6]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb,1) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,R2a,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b'
    rHat[t] = (x[t,:]*b)[1]           #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])        #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

println("MSE of AR(1) model and the benchmark: ", round([MSE_Model MSE_Bench],2))
println("out-of-sample R2: ", round(1-MSE_Model/MSE_Bench,3))

MSE of AR(1) model and the benchmark: [1.25 1.24]
out-of-sample R2: -0.002


In [7]:
using Gadfly

YearFrac  = Dates.year(dN) + (Dates.month(dN)-1)/12 + (Dates.day(dN)-1)/(31*12)
plot1 = plot(x=YearFrac[2:end],y=bT[:,1],Geom.line,Theme(default_color=colorant"blue"),
Scale.x_continuous(minvalue=1979,maxvalue=2016),
Guide.xticks(ticks=[1980,1990,2000,2010]),
Guide.title("Estimated intercept"),
Guide.ylabel(" "))
display(plot1)

plot2 = plot(x=YearFrac[2:end],y=bT[:,2],Geom.line,Theme(default_color=colorant"red"),
Scale.x_continuous(minvalue=1979,maxvalue=2016),
Guide.xticks(ticks=[1980,1990,2000,2010]),
Guide.title("Estimated AR(1) slope"),
Guide.ylabel(" "))
display(plot2)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 x 
 
 
 1980 
 1990 
 2000 
 2010 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M20.96,61.52 L 20.99 59.39 21 61.04 21.01 68.64 21.02 68.45 21.04 67.84 21.05 66.84 21.06 57.22 21.07 53.58 21.08 50.88 21.1 53.49 21.11 50.39 21.12 44.53 21.13 48.64 21.13 49.42 21.16 50.39 21.17 54.26 21.18 54.01 21.18 53.9 21.19 51.07 21.22 48.09 21.23 51.97 21.23 54.88 21.24 50.68 21.25 48.05 21.28 47.91 21.29 53.9 21.31 52.88 21.32 51.05 21.34 44.41 21.35 40.91 21.36 43.04 21.37 46.6 21.38 52.54 21.4 54.48 21.41 51.83 21.42 57.31 21.43 57.75 21.43 58.28 21.46 57.16 21.47 58.73 21.48 56.53 21.48 50.94 21.49 51.62 21.52 51.82 21.53 51.74 21.53 48.42 21.54 47.22 21.55 47.99 21.58 48.47 21.58 47.28 21.59 40.91 21.6 40.43 21.61 43.2 21.63 38.25 21.64 34.6 21.65 34.94 21.66 31.8 21.67 33.09 21.69 32.27 21.7 30.26 21.71 30.47 21.72 30.44 21.72 32.42 21.75 32.67 21.76 30.46 21.77 31.47 21.77 31.7 21.78 31.99 21.82 30.94 21.82 39.15 21.83 44.29 21.84 41.53 21.87 38.64 21.87 37.31 21.88 40.4 21.89 38.77 21.9 39.03 21.92 35.42 21.93 35.75 21.94 39.48 21.95 37.93 21.96 29.47 21.98 30.42 21.99 34.15 22 33.66 22.01 32.83 22.02 32.24 22.05 35.95 22.06 39.19 22.07 34.49 22.07 35.06 22.08 33.05 22.11 29.67 22.12 34.51 22.12 49.22 22.13 53.99 22.14 54.21 22.17 56.39 22.17 60.72 22.18 60.66 22.19 59.83 22.2 59.23 22.22 67.06 22.23 69.43 22.24 70.54 22.25 69.61 22.26 71.47 22.28 68.92 22.29 68.85 22.3 61.58 22.31 65.66 22.32 62.23 22.34 62.94 22.35 65.53 22.36 67.54 22.37 72.27 22.37 69.74 22.4 65.66 22.41 59.62 22.42 62.62 22.42 60.69 22.43 58.5 22.46 60.13 22.47 58.45 22.47 60.58 22.49 59.64 22.51 57.2 22.52 50.91 22.53 53.16 22.54 51.72 22.55 51.9 22.58 54.15 22.59 55.04 22.6 51.76 22.61 50.96 22.61 49 22.64 50.89 22.65 50.27 22.66 51.02 22.66 50.95 22.67 50.61 22.7 46.98 22.71 46.54 22.71 49.85 22.72 49.78 22.73 49.66 22.76 51.83 22.77 51.38 22.78 51.16 22.79 50.8 22.81 51.34 22.83 51.09 22.84 57.64 22.85 58.46 22.87 54.22 22.88 54.07 22.89 48.15 22.9 48.81 22.91 46.59 22.93 46.96 22.94 45.81 22.95 44.09 22.96 44.74 22.96 45.81 22.99 44.73 23 42.29 23.01 44.22 23.01 38.56 23.02 38.72 23.05 39.19 23.06 36.01 23.06 38.27 23.07 34.95 23.08 37.73 23.11 34.78 23.11 36.88 23.12 36.01 23.13 33.63 23.14 32.8 23.16 29.21 23.17 31.34 23.18 29.19 23.19 28.34 23.2 32.51 23.23 36.04 23.24 38.08 23.25 32.99 23.26 35.77 23.28 36.29 23.29 40.65 23.3 38.48 23.3 42.38 23.31 42.23 23.35 39.14 23.36 41.94 23.37 41.09 23.38 45.14 23.39 51.81 23.41 56.34 23.42 56.97 23.43 53.66 23.44 56.14 23.45 59.24 23.47 59.45 23.48 67.48 23.49 61.27 23.5 61.18 23.5 64.23 23.53 66.18 23.54 73.94 23.55 73.44 23.55 74.76 23.56 75.85 23.59 69.3 23.6 66.5 23.6 66.67 23.61 65.5 23.65 66.95 23.65 71.77 23.66 68.62 23.67 64.28 23.68 62.58 23.7 63.55 23.71 65.82 23.72 66.12 23.73 68.75 23.74 69.7 23.76 70.82 23.77 72.55 23.78 63.37 23.79 63.58 23.8 62.15 23.82 60.64 23.83 59.8 23.84 59.47 23.85 58.62 23.86 60.61 23.89 60.19 23.9 58.48 23.9 58.99 23.91 56.94 23.92 59.41 23.95 62.49 23.95 62.06 23.96 58.75 23.97 58 23.98 57.89 24 57.2 24.01 56.67 24.02 56.91 24.03 56.74 24.04 54.1 24.07 51.29 24.08 50.24 24.09 49.21 24.09 52.91 24.12 50.38 24.13 51.57 24.14 52.07 24.14 47.78 24.15 47.94 24.18 47.24 24.19 46.43 24.19 44.82 24.2 42.62 24.21 43.83 24.24 43.26 24.24 42.89 24.25 43.15 24.26 42.8 24.27 45.89 24.29 46.9 24.3 45.98 24.31 45 24.32 42.37 24.33 43.61 24.35 43.96 24.37 47.3 24.38 45.6 24.39 44.48 24.42 41.6 24.43 40.67 24.44 41.67 24.44 41.43 24.45 43.36 24.48 41.56 24.49 38.2 24.49 39.67 24.5 39.04 24.51 36.14 24.54 35.63 24.54 35.08 24.55 35.79 24.56 36.29 24.57 36.59 24.59 38.39 24.6 37.13 24.61 35.8 24.62 36.31 24.63 37.33 24.65 38.13 24.66 38.55 24.67 39.01 24.68 37.71 24.69 35.23 24.71 35.16 24.72 33.49 24

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 x 
 
 
 1980 
 1990 
 2000 
 2010 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M22.06,48.91 L 22.09 48.9 22.1 49.25 22.11 48.01 22.12 48.15 22.14 48.18 22.15 48.12 22.16 47.13 22.17 42.95 22.18 41.49 22.2 42.45 22.21 43.26 22.22 41.45 22.23 44.71 22.23 44.38 22.26 44.25 22.27 43.77 22.28 43.88 22.28 43.88 22.29 43.87 22.32 42.8 22.32 44.5 22.33 43.18 22.34 44.74 22.35 43.42 22.38 43.36 22.39 43.64 22.41 44.37 22.42 44.29 22.44 42.6 22.45 38.71 22.46 40.25 22.46 39.34 22.47 36.61 22.5 35.22 22.51 36.07 22.51 37.83 22.52 37.53 22.53 37.45 22.56 37.56 22.56 37.78 22.57 38.27 22.58 36.59 22.59 37.29 22.61 37.29 22.62 37.29 22.63 37.28 22.64 36.62 22.65 36.82 22.67 36.77 22.68 36.88 22.69 35.73 22.7 35.09 22.7 35.72 22.73 37.81 22.74 34.86 22.74 35.17 22.75 35.1 22.76 35.88 22.79 36.03 22.79 35.81 22.8 35.89 22.81 35.89 22.82 35.89 22.84 35.8 22.85 36 22.86 36.42 22.87 36.39 22.88 36.37 22.91 36.44 22.92 38.04 22.93 31.87 22.93 34 22.96 33.02 22.97 32.1 22.98 33.3 22.98 34.16 22.99 34.21 23.02 34.32 23.02 34.58 23.03 34.6 23.04 35.6 23.05 34.08 23.07 36.68 23.08 36.34 23.09 36.64 23.1 36.65 23.11 36.55 23.14 37.06 23.15 34.98 23.16 37.65 23.16 38.16 23.17 38.27 23.2 36.83 23.21 41.43 23.21 28.4 23.22 20.58 23.23 20.4 23.26 19.96 23.26 18.23 23.27 18.27 23.28 18.39 23.29 18.32 23.31 19.35 23.32 16.38 23.33 15.71 23.34 16 23.35 16.19 23.37 17 23.38 16.97 23.39 16.35 23.4 22.58 23.4 24.52 23.43 24.88 23.44 24.73 23.44 23.78 23.45 21.77 23.46 23.97 23.49 22.79 23.49 17.6 23.5 22.03 23.51 22.75 23.52 22.14 23.54 22.92 23.55 23.31 23.56 23.88 23.58 24.22 23.6 23.95 23.61 20.76 23.62 24.34 23.63 24.74 23.63 24.78 23.67 24.77 23.68 24.4 23.68 25.15 23.69 24.65 23.7 24.14 23.72 25.01 23.73 25.2 23.74 25.25 23.75 25.26 23.76 25.26 23.78 25.03 23.79 24.66 23.8 25.41 23.81 25.45 23.82 25.46 23.84 25.47 23.86 25.66 23.86 25.66 23.87 25.64 23.9 25.69 23.91 25.71 23.92 25.92 23.93 24.96 23.96 26.44 23.96 26.32 23.97 25.39 23.98 26.42 23.99 26.36 24.01 26.55 24.02 26.57 24.03 26.14 24.04 26.44 24.05 26.34 24.07 26.59 24.08 26.13 24.09 27.36 24.1 29.4 24.1 29.62 24.13 29.67 24.14 29.96 24.14 31.7 24.15 33.16 24.16 35.07 24.19 36.71 24.19 37.88 24.2 38.24 24.21 37.94 24.22 37.42 24.24 36.4 24.25 38.72 24.26 39.69 24.27 39.33 24.28 40.43 24.31 37.6 24.32 36.27 24.33 38.52 24.33 41.92 24.36 41.61 24.37 40.9 24.38 42.66 24.38 43.97 24.39 44.09 24.43 44.24 24.44 46.38 24.45 46.88 24.46 47.37 24.47 42.06 24.49 36.94 24.5 36.4 24.51 37.14 24.52 38.92 24.52 37.44 24.55 37.3 24.56 36.35 24.56 45.58 24.57 45.49 24.58 45.78 24.61 44.62 24.61 41.35 24.62 42.09 24.63 42.02 24.64 41.72 24.66 43.32 24.67 39.59 24.68 39.72 24.69 39.68 24.72 40.02 24.73 38.74 24.74 41.64 24.75 39.45 24.75 37.8 24.78 38.25 24.79 37.91 24.8 37.78 24.8 37.49 24.81 37 24.84 36.73 24.84 36.3 24.85 39.67 24.86 40.08 24.87 39.9 24.89 39.46 24.9 39.17 24.91 39.11 24.93 39.04 24.94 39.39 24.96 39.55 24.97 39.48 24.98 39.66 24.98 39.81 24.99 40.83 25.02 39.43 25.03 39.69 25.03 39.64 25.04 39.11 25.05 39.08 25.08 39.05 25.08 38.97 25.09 39 25.1 39.01 25.11 38.94 25.14 37.28 25.15 36.51 25.16 36.17 25.17 37.17 25.19 38.94 25.2 39.43 25.21 39.32 25.22 39.85 25.22 40.01 25.25 39.97 25.26 39.85 25.26 39.53 25.27 38.68 25.28 39.38 25.31 39.5 25.31 39.47 25.32 39.49 25.33 39.51 25.34 39.72 25.36 39.11 25.37 39.34 25.38 39.18 25.39 38.53 25.4 39.39 25.42 39.31 25.44 38.98 25.45 40.1 25.45 39.8 25.49 38.95 25.5 38.22 25.5 38.54 25.51 38.59 25.52 38.65 25.54 39.38 25.55 38.19 25.56 39.64 25.57 39.82 25.58 39.52 25.6 39.12 25.61 39.01 25.62 39.12 25.63 39.04 25.64 39.01 25.66 38.86 25.67 39.35 25.68 39.03 25.68 39.23 25.69 39.13 25.72 38.94 25.73 38.86 25.73 38.81 25.74 38.96 25.7